In [1]:
%%capture
%load_ext sql
%sql sqlite:///stores.db

In [2]:
%%sql
SELECT
    name
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///stores.db
Done.


type
customers
employees
offices
orderdetails
orders
payments
productlines
products


In [3]:
%%sql

SELECT 'Customers' AS table_name, 
       13 AS number_of_attribute,
       COUNT(*) AS number_of_row
  FROM Customers
  
UNION ALL

SELECT 'Products' AS table_name, 
       9 AS number_of_attribute,
       COUNT(*) AS number_of_row
  FROM Products

UNION ALL

SELECT 'ProductLines' AS table_name, 
       4 AS number_of_attribute,
       COUNT(*) AS number_of_row
  FROM ProductLines

UNION ALL

SELECT 'Orders' AS table_name, 
       7 AS number_of_attribute,
       COUNT(*) AS number_of_row
  FROM Orders

UNION ALL

SELECT 'OrderDetails' AS table_name, 
       5 AS number_of_attribute,
       COUNT(*) AS number_of_row
  FROM OrderDetails

UNION ALL

SELECT 'Payments' AS table_name, 
       4 AS number_of_attribute,
       COUNT(*) AS number_of_row
  FROM Payments

UNION ALL

SELECT 'Employees' AS table_name, 
       8 AS number_of_attribute,
       COUNT(*) AS number_of_row
  FROM Employees

UNION ALL

SELECT 'Offices' AS table_name, 
       9 AS number_of_attribute,
       COUNT(*) AS number_of_row
  FROM Offices;

 * sqlite:///stores.db
Done.


table_name,number_of_attribute,number_of_row
Customers,13,122
Products,9,110
ProductLines,4,7
Orders,7,326
OrderDetails,5,2996
Payments,4,273
Employees,8,23
Offices,9,7


Question 1: Which products should we order more of or less of?

In [4]:
%%sql

select od.productCode, p.productName, round((sum(od.quantityOrdered)*1.0)/p.quantityInStock) as low_stock

from Products as p
join OrderDetails as od
on od.productCode = p.productCode

group by od.productCode
order by low_stock desc

limit 10

 * sqlite:///stores.db
Done.


productCode,productName,low_stock
S24_2000,1960 BSA Gold Star DBD34,68.0
S12_1099,1968 Ford Mustang,14.0
S32_4289,1928 Ford Phaeton Deluxe,7.0
S32_1374,1997 BMW F650 ST,6.0
S18_2248,1911 Ford Town Car,2.0
S18_2795,1928 Mercedes-Benz SSK,2.0
S50_4713,2002 Yamaha YZR M1,2.0
S700_3167,F/A 18 Hornet 1/72,2.0
S72_3212,Pont Yacht,2.0
S12_1666,1958 Setra Bus,1.0


In [5]:
%%sql 

select productCode, sum(quantityOrdered * priceEach) as product_performance
 
from OrderDetails
group by productCode
order by product_performance desc
limit 10

 * sqlite:///stores.db
Done.


productCode,product_performance
S18_3232,276839.98
S12_1108,190755.86
S10_1949,190017.95999999996
S10_4698,170685.99999999997
S12_1099,161531.47999999992
S12_3891,152543.02
S18_1662,144959.90999999997
S18_2238,142530.62999999998
S18_1749,140535.60000000003
S12_2823,135767.03000000003


In [6]:
%%sql
WITH low_stock_table AS
(
SELECT od.productCode, p.productName, ROUND((SUM(od.quantityOrdered)*1.0)/p.quantityInStock) AS low_stock

  FROM Products as p
  JOIN OrderDetails as od
    ON od.productCode = p.productCode

 GROUP BY od.productCode
 ORDER BY low_stock desc
)  

SELECT productCode, SUM(quantityOrdered * priceEach) AS product_performance
 
  FROM OrderDetails
 WHERE productCode IN (SELECT productCode
                         FROM low_stock_table)
 GROUP BY productCode
 ORDER BY product_performance desc
 LIMIT 10


 * sqlite:///stores.db
Done.


productCode,product_performance
S18_3232,276839.98
S12_1108,190755.86
S10_1949,190017.95999999996
S10_4698,170685.99999999997
S12_1099,161531.47999999992
S12_3891,152543.02
S18_1662,144959.90999999997
S18_2238,142530.62999999998
S18_1749,140535.60000000003
S12_2823,135767.03000000003


Q2:how should we match marketing and communication strategies to customer behaviors?

In [7]:
%%sql
-- revenue by customer
SELECT o.customerNumber, SUM(quantityOrdered * (priceEach - buyPrice)) AS revenue
  FROM products p
  JOIN orderdetails od
    ON p.productCode = od.productCode
  JOIN orders o
    ON o.orderNumber = od.orderNumber
 GROUP BY o.customerNumber;


 * sqlite:///stores.db
Done.


customerNumber,revenue
103,10063.800000000001
112,31312.72
114,70311.06999999999
119,60875.30000000001
121,41391.52
124,236769.38999999998
128,27728.34
129,28092.43
131,58669.10000000001
141,326519.65999999986


In [8]:
%%sql

-- Top 5 VIP customers
WITH 

money_in_by_customer_table AS (
SELECT o.customerNumber, SUM(quantityOrdered * (priceEach - buyPrice)) AS revenue
  FROM products p
  JOIN orderdetails od
    ON p.productCode = od.productCode
  JOIN orders o
    ON o.orderNumber = od.orderNumber
 GROUP BY o.customerNumber
)

SELECT contactLastName, contactFirstName, city, country, mc.revenue
  FROM customers c
  JOIN money_in_by_customer_table mc
    ON mc.customerNumber = c.customerNumber
 ORDER BY mc.revenue DESC
 LIMIT 5;

 * sqlite:///stores.db
Done.


contactLastName,contactFirstName,city,country,revenue
Freyre,Diego,Madrid,Spain,326519.65999999986
Nelson,Susan,San Rafael,USA,236769.38999999998
Young,Jeff,NYC,USA,72370.09000000001
Ferguson,Peter,Melbourne,Australia,70311.06999999999
Labrune,Janine,Nantes,France,60875.30000000001


In [9]:
%%sql

 -- Top 5 less engaging customers
WITH 

money_in_by_customer_table AS (
SELECT o.customerNumber, SUM(quantityOrdered * (priceEach - buyPrice)) AS revenue
  FROM products p
  JOIN orderdetails od
    ON p.productCode = od.productCode
  JOIN orders o
    ON o.orderNumber = od.orderNumber
 GROUP BY o.customerNumber
)

SELECT contactLastName, contactFirstName, city, country, mc.revenue
  FROM customers c
  JOIN money_in_by_customer_table mc
    ON mc.customerNumber = c.customerNumber
 ORDER BY mc.revenue
 LIMIT 5;

 * sqlite:///stores.db
Done.


contactLastName,contactFirstName,city,country,revenue
Young,Mary,Glendale,USA,2610.870000000001
Taylor,Leslie,Brickhaven,USA,6586.0199999999995
Ricotti,Franco,Milan,Italy,9532.93
Schmitt,Carine,Nantes,France,10063.800000000001
Smith,Thomas,London,UK,10868.039999999999


Question 3: How much can we spend on acquiring new customers?

In [10]:
%%sql
-- Customer LTV
WITH 

money_in_by_customer_table AS (
SELECT o.customerNumber, SUM(quantityOrdered * (priceEach - buyPrice)) AS revenue
  FROM products p
  JOIN orderdetails od
    ON p.productCode = od.productCode
  JOIN orders o
    ON o.orderNumber = od.orderNumber
 GROUP BY o.customerNumber
)

SELECT AVG(mc.revenue) AS ltv
  FROM money_in_by_customer_table mc;

 * sqlite:///stores.db
Done.


ltv
39039.59438775511
